# Introductive code

In order to work properly, our notebook requires 2 folders:

+ "MaskDataset" folder, which is the result of unzipping 'artificial-neural-networks-and-deep-learning-2020.zip': we've created a cell that unzips the file, but it should be run only if "MaskDataset" folder wasn't already created

+ "MaskDatasetSorted" folder, which contains three folders (one per label) with all the images for flow_from_directory: it can be created running the three code cells below the "Sort the images for flow_from_directory method" title, and each of them should be run only 1 time (in order to avoid multiple copies of the same image)

In [ ]:
# Print all the intermediate operations (for debugging)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Import most relevant libraries

import os
import tensorflow as tf
import numpy as np


# Import library for handling json files

import json


# Import shutil for the copy in sorted folders

import shutil

In [ ]:
# Fix a seed

SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
# Add Colab (with Drive)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# If the "MaskDataset" folder wasn't already created, this cell unzips the zip file and creates "MaskDataset";
# please run this cell only if "MaskDataset" wasn't already created

!unzip '/content/drive/My Drive/ANNDL_Results/Challenge1/artificial-neural-networks-and-deep-learning-2020.zip' -d '/content/drive/My Drive/ANNDL_Results/Challenge1'

In [ ]:
# Path to the current working directory (which contains MaskDataset folder)

cwd = '/content/drive/My Drive/ANNDL_Results/Challenge1' # Working directory, with the dataset folder (we had issues with os.getcwd())

# The following variables are inherited from previous code for Kaggle: they may be 
# changed to switch to Kaggle without changing the rest of the implementation

work_dir = cwd
work_dir2 = cwd
work_dir3 = cwd

In [ ]:
# Path to the dataset folder (MaskDataset folder must be already created)

dataset_dir = os.path.join(work_dir,"MaskDataset")

# Sort the images for flow_from_directory method

Run the following 3 cells to create "MaskDatasetSorted" folder, which contains three folders (one per label) with all the images for flow_from_directory. Please run these 3 cells only if "MaskDatasetSorted" wasn't already created

In [ ]:
# From the file .json, extract a dictionary with:

# Key: name of the image
# Value: label of the image

with open(os.path.join(dataset_dir,"train_gt.json")) as json_file: 
    labels = json.load(json_file)

In [ ]:
# Create the folders for flow_from_directory method

sorted_training_dir = os.path.join(work_dir2,"MaskDatasetSorted")
if not os.path.exists(sorted_training_dir):
    os.makedirs(sorted_training_dir)

folder_0 = os.path.join(sorted_training_dir,"label0")
if not os.path.exists(folder_0):
    os.makedirs(folder_0)
    
folder_1 = os.path.join(sorted_training_dir,"label1")
if not os.path.exists(folder_1):
    os.makedirs(folder_1)

folder_2 = os.path.join(sorted_training_dir,"label2")
if not os.path.exists(folder_2):
    os.makedirs(folder_2)

In [ ]:
# Copy the images in the proper folder

training_dir = os.path.join(dataset_dir, 'training')

for name in labels.keys():
    photo_dir = os.path.join(training_dir,name)
    
    if labels[name] == 0:
        shutil.copy2(photo_dir,folder_0)
    if labels[name] == 1:
        shutil.copy2(photo_dir,folder_1)
    if labels[name] == 2:
        shutil.copy2(photo_dir,folder_2)

# Set all the necessary variables in ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input 

apply_data_augmentation = True

# Create training ImageDataGenerator object (for data augmentation and for transfer learning)

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=7,
                                        height_shift_range=7,
                                        zoom_range=0.15,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='constant',
                                        cval=0,
                                        validation_split = 0.08,
                                        preprocessing_function=preprocess_input)  # to apply vgg normalization
else:
    train_data_gen = ImageDataGenerator(validation_split = 0.08,
                                        preprocessing_function=preprocess_input)

# Create train and validation generators

In [ ]:
# Path to the training folder (data will be splitted, using ImageDataGenerator objects with flow_from_directory method)

training_dir = os.path.join(work_dir2, 'MaskDatasetSorted')

In [ ]:
# Some useful parameters

# Batch size
bs = 32

# img shape
img_h = 360
img_w = 360

# Number of classes
num_classes = 3

In [ ]:
# Create generators to read images from MaskDatasetSorted (created before)

decide_class_indices = True
if decide_class_indices:
    classes = ['label0',    # 0
               'label1',    # 1
               'label2']    # 2
else:
    classes=None

# Training
    
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h, img_w),
                                               color_mode="rgb",
                                               classes=classes,
                                               class_mode='categorical',
                                               batch_size=bs,
                                               shuffle=True,
                                               seed=SEED,
                                               subset='training')

# Validation

valid_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h, img_w),
                                               color_mode="rgb",
                                               classes=classes,
                                               class_mode='categorical',
                                               batch_size=bs,
                                               shuffle=True,
                                               seed=SEED,
                                               subset='validation')

Found 5166 images belonging to 3 classes.
Found 448 images belonging to 3 classes.


In [ ]:
# Check how keras assigned the labels
train_gen.class_indices

{'label0': 0, 'label1': 1, 'label2': 2}

In [ ]:
# Create Dataset objects both for training and for validation with tf.data.Dataset.from_generator


# Training

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()


# Validation

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

# Build and fit the model

In [ ]:
# Load VGG16 Model

vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

# Create Model

model = tf.keras.Sequential()

# Create the convolutional network for transfer learning (or fine tuning)

finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False

model.add(vgg)

# Set the fully connected layers for classification

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=224, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
# Visualize created model as a table

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 11, 11, 512)       14714688  
_________________________________________________________________
flatten (Flatten)            (None, 61952)             0         
_________________________________________________________________
dropout (Dropout)            (None, 61952)             0         
_________________________________________________________________
dense (Dense)                (None, 224)               13877472  
_________________________________________________________________
dense_1 (Dense)              (None, 32)                7200      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 28,599,459
Trainable params: 20,964,195
Non-trainable params: 7,635,264
____________________________________

In [ ]:
# Optimization params

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()


# Learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)


# Validation metrics
metrics = ['accuracy']


# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

We performed a training of 25 epochs on the model. Differently from our best transfer learning - based network, training seems to start also in the very first run and doesn't get stuck at 33%.

The following cells are taken from the lab session, they will also save checkpoints: if you don't want checkpoints please comment the lines  'callbacks.append(ckpt_callback)' in each cell.

In [ ]:
import os
from datetime import datetime

cwd = cwd  # use your local directory if you are not using Drive

exps_dir = os.path.join(cwd, 'classification_experiments_')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_name = 'Fine_tuning'

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), save_best_only=True,
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Early Stopping

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

# Fit the model (few epochs to check the validity of the model)

model.fit(x=train_dataset,
          epochs=25,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/25
162/162 [==============================] - 2792s 17s/step - loss: 1.5573 - accuracy: 0.5474 - val_loss: 0.9804 - val_accuracy: 0.7232
Epoch 2/25
162/162 [==============================] - 305s 2s/step - loss: 0.9687 - accuracy: 0.7338 - val_loss: 0.7987 - val_accuracy: 0.8348
Epoch 3/25
162/162 [==============================] - 300s 2s/step - loss: 0.8667 - accuracy: 0.7795 - val_loss: 0.8064 - val_accuracy: 0.8125
Epoch 4/25
162/162 [==============================] - 307s 2s/step - loss: 0.7681 - accuracy: 0.8337 - val_loss: 0.7046 - val_accuracy: 0.8661
Epoch 5/25
162/162 [==============================] - 309s 2s/step - loss: 0.7182 - accuracy: 0.8436 - val_loss: 0.6964 - val_accuracy: 0.8705
Epoch 6/25
162/162 [==============================] - 308s 2s/step - loss: 0.6876 - accuracy: 0.8653 - val_loss: 0.6414 - val_accuracy: 0.8839
Epoch 7/25
162/162 [==============================] - 309s 2s/step - loss: 0.6474 - accuracy: 0.8779 - val_loss: 0.5989 - val_accuracy: 0.90

We tried 15 more epochs, starting from the best model of the previous 25: validation accuracy is stable around 92%, but the model seems unable to improve. Being unable to further refine our result, we didn't submit new predictions, and we interrupted our trainings.

In [ ]:
dir_with_weights = '/content/drive/My Drive/ANNDL_Results/Challenge1/Checkpoints/Transfer_model_weights'
dir_with_weights

model.load_weights(os.path.join(dir_with_weights, 'TransferCheckpoint.ckpt'))

'/content/drive/My Drive/ANNDL_Results/Challenge1/Checkpoints/Transfer_model_weights'

In [ ]:
import os
from datetime import datetime

cwd = cwd  # use your local directory if you are not using Drive

exps_dir = os.path.join(cwd, 'classification_experiments_')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_name = 'Fine_tuning_2'

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), save_best_only=True,
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Early Stopping

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

# Fit the model

model.fit(x=train_dataset,
          epochs=15,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/15
162/162 [==============================] - 1183s 7s/step - loss: 0.2708 - accuracy: 0.9743 - val_loss: 0.4637 - val_accuracy: 0.9375
Epoch 2/15
162/162 [==============================] - 216s 1s/step - loss: 0.2718 - accuracy: 0.9708 - val_loss: 0.4729 - val_accuracy: 0.9196
Epoch 3/15
162/162 [==============================] - 215s 1s/step - loss: 0.2635 - accuracy: 0.9692 - val_loss: 0.5200 - val_accuracy: 0.9152
Epoch 4/15
162/162 [==============================] - 214s 1s/step - loss: 0.2508 - accuracy: 0.9739 - val_loss: 0.4934 - val_accuracy: 0.9152
Epoch 5/15
162/162 [==============================] - 213s 1s/step - loss: 0.2272 - accuracy: 0.9787 - val_loss: 0.4643 - val_accuracy: 0.9286
Epoch 6/15
162/162 [==============================] - 213s 1s/step - loss: 0.2313 - accuracy: 0.9760 - val_loss: 0.4769 - val_accuracy: 0.9241
Epoch 7/15
162/162 [==============================] - 212s 1s/step - loss: 0.1987 - accuracy: 0.9851 - val_loss: 0.6257 - val_accuracy: 0.886

# Test the model, generate the csv file

In [ ]:
# Import necessary libraries

import os

from datetime import datetime

from PIL import Image

In [ ]:
# Given function for saving the csv file, once the experiment is complete

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
# our version

from keras.applications.vgg16 import preprocess_input

test_dir = os.path.join(dataset_dir, 'test')
image_filenames = next(os.walk(test_dir))[2]

results = {}

# Iterate on each image
for image_name in image_filenames:

   # Open image
   img = Image.open(os.path.join(test_dir, image_name)).convert('RGB')

   # Create a tensor from each image

   img_array = np.array(img.resize((img_w,img_h)))
   img_array = np.expand_dims(img_array, 0) 

   # Normalize, predict and add to the dictionary
   softmax = model.predict(x = preprocess_input(img_array))
   prediction = tf.argmax(softmax,1)
   results[image_name] = int(prediction)

# Create csv file with the given function
create_csv(results,work_dir3)